# Creating LLM-Powered A2A Agents

This notebook demonstrates how to create and use agents powered by large language models (LLMs) like OpenAI's GPT and Anthropic's Claude with the Python A2A package. We'll explore:

1. Setting up LLM-powered agents
2. Customizing agent behavior with system prompts
3. Comparing responses from different LLM providers
4. Implementing function calling capabilities

Let's get started!

## Setup

First, let's make sure we have the Python A2A package installed with the necessary dependencies:

In [ ]:
!pip install "python-a2a[openai,anthropic]"

Now, let's import the necessary components from the package:

In [ ]:
import os
from python_a2a import (
    Message, TextContent, FunctionCallContent, FunctionResponseContent, FunctionParameter,
    MessageRole, Conversation,
    OpenAIA2AServer, AnthropicA2AServer,
    pretty_print_message, pretty_print_conversation
)

## Setting Up API Keys

To use the LLM-powered agents, we need API keys for the respective providers. You should set these as environment variables or enter them here:

In [ ]:
# Set your API keys here (or use environment variables)
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"

# Get API keys from environment
openai_api_key = os.environ.get("OPENAI_API_KEY")
anthropic_api_key = os.environ.get("ANTHROPIC_API_KEY")

# Check if API keys are available
if not openai_api_key:
    print("⚠️ OpenAI API key not found. OpenAI examples will not work.")
else:
    print("✅ OpenAI API key found.")
    
if not anthropic_api_key:
    print("⚠️ Anthropic API key not found. Claude examples will not work.")
else:
    print("✅ Anthropic API key found.")

## Creating LLM-Powered Agents

### OpenAI-Powered Agent

Let's start by creating an agent powered by OpenAI's models:

In [ ]:
# Skip this cell if OpenAI API key is not available
if openai_api_key:
    # Create an OpenAI-powered agent
    openai_agent = OpenAIA2AServer(
        api_key=openai_api_key,
        model="gpt-4",  # You can use "gpt-3.5-turbo" for faster, cheaper responses
        temperature=0.7,
        system_prompt="You are a helpful assistant that provides clear, concise answers."
    )
    print("OpenAI agent created successfully!")

### Anthropic-Powered Agent

Now, let's create an agent powered by Anthropic's Claude models:

In [ ]:
# Skip this cell if Anthropic API key is not available
if anthropic_api_key:
    # Create an Anthropic-powered agent
    claude_agent = AnthropicA2AServer(
        api_key=anthropic_api_key,
        model="claude-3-opus-20240229",  # You can use other Claude models too
        temperature=0.7,
        max_tokens=1000,
        system_prompt="You are a helpful assistant that provides clear, concise answers."
    )
    print("Claude agent created successfully!")

## Testing the Agents

Let's create a function to test our agents with different prompts:

In [ ]:
def test_agent(agent, prompt, agent_name="Agent"):
    """Test an agent with a prompt and print the response."""
    print(f"🔍 Testing {agent_name}...")
    
    # Create a message with the prompt
    message = Message(
        content=TextContent(text=prompt),
        role=MessageRole.USER
    )
    
    # Get a response from the agent
    response = agent.handle_message(message)
    
    print(f"\n🤖 {agent_name} response:\n")
    print(response.content.text)
    print("\n" + "-"*80 + "\n")
    
    return response

Now, let's test our agents with a few different prompts:

In [ ]:
# Test OpenAI agent
if openai_api_key:
    openai_response = test_agent(
        openai_agent, 
        "Explain the concept of agent interoperability and why it's important for AI systems.",
        "OpenAI GPT-4"
    )

In [ ]:
# Test Claude agent
if anthropic_api_key:
    claude_response = test_agent(
        claude_agent, 
        "Explain the concept of agent interoperability and why it's important for AI systems.",
        "Anthropic Claude"
    )

## Customizing Agent Behavior with System Prompts

The behavior of LLM-powered agents can be customized using system prompts. Let's create specialized agents for different purposes:

In [ ]:
# Create specialized OpenAI agents (if API key is available)
if openai_api_key:
    # Technical documentation agent
    technical_agent = OpenAIA2AServer(
        api_key=openai_api_key,
        model="gpt-4",
        temperature=0.2,  # Lower temperature for more deterministic responses
        system_prompt=(
            "You are a technical documentation specialist. "
            "Provide detailed, accurate explanations of technical concepts with examples where appropriate. "
            "Use markdown formatting for clarity and structure."
        )
    )
    
    # Creative writing agent
    creative_agent = OpenAIA2AServer(
        api_key=openai_api_key,
        model="gpt-4",
        temperature=0.9,  # Higher temperature for more creative responses
        system_prompt=(
            "You are a creative writer and storyteller. "
            "Generate engaging, imaginative content with rich descriptions and compelling narratives. "
            "Be original and think outside the box."
        )
    )
    
    print("Specialized OpenAI agents created successfully!")

Now, let's compare the responses from our specialized agents on the same prompt:

In [ ]:
# Test specialized agents (if API key is available)
if openai_api_key:
    prompt = "Write about artificial intelligence agents working together."
    
    technical_response = test_agent(technical_agent, prompt, "Technical Documentation Agent")
    creative_response = test_agent(creative_agent, prompt, "Creative Writing Agent")

## Implementing Function Calling Capabilities

One powerful feature of OpenAI's models is function calling, which allows the model to request the execution of specific functions. Let's implement a weather function for our OpenAI agent:

In [ ]:
# Define a weather function
def get_weather(location, unit="celsius"):
    """Simulate getting weather data for a location."""
    # In a real application, this would call a weather API
    # For this example, we'll use mock data
    weather_data = {
        "New York": {"temperature": 22, "conditions": "Partly Cloudy", "humidity": 65, "wind_speed": 8},
        "London": {"temperature": 18, "conditions": "Rainy", "humidity": 80, "wind_speed": 12},
        "Tokyo": {"temperature": 26, "conditions": "Sunny", "humidity": 70, "wind_speed": 5},
        "Sydney": {"temperature": 28, "conditions": "Clear", "humidity": 55, "wind_speed": 10},
        "Paris": {"temperature": 20, "conditions": "Cloudy", "humidity": 60, "wind_speed": 7}
    }
    
    # Get the weather data for the location (default to New York if not found)
    location_data = weather_data.get(location, weather_data["New York"])
    
    # Convert temperature if necessary
    if unit.lower() == "fahrenheit":
        location_data["temperature"] = location_data["temperature"] * 9/5 + 32
        location_data["unit"] = "fahrenheit"
    else:
        location_data["unit"] = "celsius"
    
    # Add the location to the data
    location_data["location"] = location
    
    return location_data

# Define the function schema for OpenAI
weather_function = {
    "name": "get_weather",
    "description": "Get the current weather for a location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. New York"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "The temperature unit"
            }
        },
        "required": ["location"]
    }
}

# Create a function-enabled OpenAI agent (if API key is available)
if openai_api_key:
    function_agent = OpenAIA2AServer(
        api_key=openai_api_key,
        model="gpt-4",
        temperature=0.7,
        system_prompt="You are a helpful assistant that can provide weather information.",
        functions=[weather_function]
    )
    print("Function-enabled OpenAI agent created successfully!")

# Now let's create a function to test the function calling capability
def test_function_calling(agent, prompt, agent_name="Agent"):
    """Test an agent's function calling capability with a prompt."""
    print(f"🔍 Testing {agent_name} function calling...")
    
    # Create a message with the prompt
    message = Message(
        content=TextContent(text=prompt),
        role=MessageRole.USER
    )
    
    # Get a response from the agent
    response = agent.handle_message(message)
    
    # Check if the response is a function call
    if response.content.type == "function_call":
        print(f"\n🤖 {agent_name} wants to call a function:\n")
        print(f"Function: {response.content.name}")
        print("Parameters:")
        for param in response.content.parameters:
            print(f"  {param.name}: {param.value}")
        
        # Execute the function
        if response.content.name == "get_weather":
            params = {p.name: p.value for p in response.content.parameters}
            result = get_weather(**params)
            
            # Create a function response
            function_response = Message(
                content=FunctionResponseContent(
                    name="get_weather",
                    response=result
                ),
                role=MessageRole.AGENT,
                parent_message_id=response.message_id,
                conversation_id=response.conversation_id
            )
            
            print(f"\n⚙️ Function result:\n")
            print(result)
            
            # Get the final response from the agent
            final_response = agent.handle_message(function_response)
            
            print(f"\n🤖 {agent_name} final response:\n")
            print(final_response.content.text)
            
            return final_response
    else:
        print(f"\n🤖 {agent_name} response (no function call):\n")
        print(response.content.text)
        
    print("\n" + "-"*80 + "\n")
    
    return response

# Test the function calling capability (if API key is available)
if openai_api_key:
    # Test with a prompt that should trigger function calling
    function_response = test_function_calling(
        function_agent,
        "What's the weather like in Tokyo right now?",
        "Function-Enabled Agent"
    )
    
    # Test with another location
    function_response2 = test_function_calling(
        function_agent,
        "I'm planning a trip to London. What's the weather there?",
        "Function-Enabled Agent"
    )

## Working with Conversations

LLM-powered agents can also handle full conversations with message history. Let's create a function to test this:

In [ ]:
def test_conversation(agent, prompts, agent_name="Agent"):
    """Test an agent with a series of prompts in a conversation."""
    print(f"🔍 Testing {agent_name} conversation...")
    
    # Create a new conversation
    conversation = Conversation()
    
    # Add each prompt to the conversation and get responses
    for i, prompt in enumerate(prompts):
        print(f"\n📝 User message {i+1}: {prompt}")
        
        # Add the user message to the conversation
        conversation.create_text_message(
            text=prompt,
            role=MessageRole.USER
        )
        
        # Send the conversation to the agent
        updated_conversation = agent.handle_conversation(conversation)
        
        # Update our conversation reference
        conversation = updated_conversation
        
        # Print the agent's response
        response = conversation.messages[-1]
        print(f"\n🤖 {agent_name} response {i+1}:\n")
        print(response.content.text)
        print("\n" + "-"*50 + "\n")
    
    return conversation

Now, let's test the conversation handling capability:

In [ ]:
# Test conversation handling (if API key is available)
if openai_api_key:
    # Series of prompts for a conversation
    prompts = [
        "What is the A2A protocol?",
        "How does it relate to other agent frameworks?",
        "Can you give an example of how it might be used in a real application?"
    ]
    
    conversation = test_conversation(openai_agent, prompts, "OpenAI Agent")

Let's test the Claude agent as well (if available):

In [ ]:
# Test Claude agent conversation (if API key is available)
if anthropic_api_key:
    # Series of prompts for a conversation
    prompts = [
        "What is the A2A protocol?",
        "How does it relate to other agent frameworks?",
        "Can you give an example of how it might be used in a real application?"
    ]
    
    claude_conversation = test_conversation(claude_agent, prompts, "Claude Agent")

## Conclusion

In this notebook, we've learned how to:

1. Create LLM-powered A2A agents using OpenAI and Anthropic models
2. Customize agent behavior using system prompts
3. Implement function calling capabilities
4. Handle multi-turn conversations

The Python A2A package makes it easy to create powerful, interoperable agents that can leverage the capabilities of different LLM providers. By standardizing the message format and providing a consistent interface, A2A enables seamless communication between agents regardless of the underlying model or implementation.

This interoperability is key to building modular AI systems where specialized agents can collaborate to solve complex problems. Whether you're using OpenAI's GPT models, Anthropic's Claude, or other LLMs, the A2A protocol provides a common language for agent communication.